<a href="https://colab.research.google.com/github/aakashagarwal6898/Ocular/blob/master/Ocular_GCP_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install google-cloud-vision
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="YOUR_GCP_CREDENTIALS"

In [0]:
from google.cloud import vision
import io
import os
import json
import spacy

In [0]:
model_dir = "path/to/saved_model"
nlpModel = spacy.load(model_dir)
client = vision.ImageAnnotatorClient()

In [0]:
def vision_ocr(input_img):
  file = input_img
  with io.open(file, 'rb') as image_file:
    content = image_file.read()
    image = vision.types.Image(content=content)
    text_response = client.text_detection(image=image)
    texts = [text.description for text in text_response.text_annotations]
    textToPredict = texts[0]
    
  return textToPredict


def ner_predict(textToPredict):
  doc = nlpModel(textToPredict)
  max_amt = 0
  i = 1
  data = {}
  items_list = []
  # Iterating over every entitiy to create a dictionary
  for ent in doc.ents:
    # Saving only one instance of Total Bill Amount
    if (ent.label_ == "Total bill amount"):
      try:
        amt = float(ent.text)
        if amt > max_amt:
          data["Total bill amount"] = amt
      except Exception as e:
        pass
    # Creating a list of Items
    elif (ent.label_ == "Items"):
      try:
        items_list.append(ent.text)
      except Exception as e:
        print(e)
    # Checking if the detected key is already present in the key,
    # If yes then we create a new key to store that value instead of overwriting the previous one
    else:
      if ent.label_ in data.keys():
        data[ent.label_+"-"+str(i)] = ent.text
        i +=1
      else:
        data[ent.label_] = ent.text
  # Staring the list of items using the Items key in the dictionary
  data["Items"]=items_list
  # Sorting all the elements of the dictionary
  data = dict(sorted(data.items()))
  json_data = json.dumps(data, indent=2)

  return json_data

def get_prediction(input_img):
  textToPredict = vision_ocr(input_img)
  json_data = ner_predict(textToPredict)

  return json_data







In [20]:
json_data = get_prediction("path/to/input_image")
print(json_data)

{
  "Date": "02/02/19",
  "Items": [
    "Scallion Pancake",
    "Little Juicy Pork Buns",
    "Beef w Bak Choy (L)",
    "Hunan Beef (L)"
  ],
  "Store address": "1902 Jericho Turnpike\nNew Hyde Park NY 11040",
  "Store name": "Chef Wang",
  "Time": "02:14 PM"
}
